In [1]:
import random

In [2]:
class Matrix(list):
    
    @classmethod
    def zeros(cls, shape):
        n_rows, n_cols = shape
        return cls([[0] * n_cols for i in range(n_rows)])
    
    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                     for j in range(n_cols)])
        return M
    
    @property
    def shape(self):
        return ((0, 0) if not self else
               (len(self), len(self[0])))
    
    def transpose(self):
        Mt = self.zeros(self.shape)
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                Mt[i][j] = self[j][i]
        return Mt

In [3]:
def matrix_product(X, Y):
    """Computes the matrix product of X and Y
    >>> X = Matrix([[1], [2], [3]])
    >>> Y = Matrix([[4, 5, 6]])
    >>> matrix_product(X, Y)
    [[4, 5, 6], [8, 10, 12], [12, 15, 18]]
    >>> matrix_product(Y, X)
    [[32]]
    """
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    # we believe dimensions are ok
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z

In [4]:
import timeit

Garbage collector is turned off during `timeit` run

In [5]:
setup = """
shape = 64, 64
X = Matrix.random(shape)
Y = Matrix.random(shape)
"""

In [6]:
timeit.timeit("matrix_product(X, Y)", setup, number=10, globals=globals())

0.5968530519967317

In [7]:
shape = 64, 64
X, Y = Matrix.random(shape), Matrix.random(shape)

In [8]:
%timeit -n 1 matrix_product(X, Y)

63 ms ± 5.79 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
def bench(shape=(64, 64), n_iter=16):
    X = Matrix.random(shape)
    Y = Matrix.random(shape)
    for _ in range(n_iter):
        matrix_product(X, Y)

In [10]:
%time bench()

CPU times: user 997 ms, sys: 0 ns, total: 997 ms
Wall time: 997 ms


In [11]:
import cProfile

In [12]:
source = open("faster_python_faster.py").read()

In [13]:
cProfile.run(source, sort="tottime")

         41378 function calls in 0.965 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       16    0.949    0.059    0.949    0.059 <string>:32(matrix_product)
     8192    0.005    0.000    0.011    0.000 random.py:174(randrange)
     8192    0.004    0.000    0.006    0.000 random.py:224(_randbelow)
     8192    0.002    0.000    0.013    0.000 random.py:218(randint)
      128    0.002    0.000    0.015    0.000 <string>:15(<listcomp>)
     8202    0.001    0.000    0.001    0.000 {method 'getrandbits' of '_random.Random' objects}
        1    0.001    0.001    0.965    0.965 {built-in method builtins.exec}
     8192    0.000    0.000    0.000    0.000 {method 'bit_length' of 'int' objects}
        1    0.000    0.000    0.965    0.965 <string>:77(bench)
        2    0.000    0.000    0.016    0.008 <string>:11(random)
       16    0.000    0.000    0.000    0.000 <string>:9(<listcomp>)
        1    0.000    0.000    0.9

In [14]:
%load_ext line_profiler

In [15]:
from faster_python_faster import matrix_product, bench

In [16]:
%lprun -f matrix_product bench(matrix_product)

Timer unit: 1e-06 s

Total time: 3.58053 s
File: /home/leo/ME/computer-science/CSC Python/faster_python_faster.py
Function: matrix_product at line 32

Line #      Hits         Time  Per Hit   % Time  Line Contents
    32                                           def matrix_product(X, Y):
    33                                               """Computes the matrix product of X and Y
    34                                               >>> X = Matrix([[1], [2], [3]])
    35                                               >>> Y = Matrix([[4, 5, 6]])
    36                                               >>> matrix_product(X, Y)
    37                                               [[4, 5, 6], [8, 10, 12], [12, 15, 18]]
    38                                               >>> matrix_product(Y, X)
    39                                               [[32]]
    40                                               """
    41        16         64.0      4.0      0.0      n_xrows, n_xcols = X.shape
    4

`list.__getitem__` does not come for free. Let's memorize `X[i]` and `Z[i]` and change order of cycles to reduce the number of addresses.

In [17]:
def matrix_product2(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        Xi, Zi = X[i], Z[i]
        for k in range(n_ycols):
            acc = 0
            for j in range(n_xcols):
                acc += Xi[j] * Y[j][k]
            Zi[k] = acc
    return Z

In [18]:
%timeit matrix_product2(X, Y)

30 ms ± 283 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
from faster_python_faster import matrix_product2

In [20]:
%lprun -f matrix_product2 bench(matrix_product2)

Timer unit: 1e-06 s

Total time: 3.23025 s
File: /home/leo/ME/computer-science/CSC Python/faster_python_faster.py
Function: matrix_product2 at line 52

Line #      Hits         Time  Per Hit   % Time  Line Contents
    52                                           def matrix_product2(X, Y):
    53        16         38.0      2.4      0.0      n_xrows, n_xcols = X.shape
    54        16         16.0      1.0      0.0      n_yrows, n_ycols = Y.shape
    55        16        255.0     15.9      0.0      Z = Matrix.zeros((n_xrows, n_ycols))
    56      1040        331.0      0.3      0.0      for i in range(n_xrows):
    57      1024        493.0      0.5      0.0          Xi, Zi = X[i], Z[i]
    58     66560      21086.0      0.3      0.7          for k in range(n_ycols):
    59     65536      19863.0      0.3      0.6              acc = 0
    60   4259840    1308349.0      0.3     40.5              for j in range(n_xcols):
    61   4194304    1858086.0      0.4     57.5                  ac

In [21]:
def matrix_product2(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        Xi, Zi = X[i], Z[i]
        for k in range(n_ycols):
            Z[i][k] = sum(Xi[j] * Y[j][k] for j in range(n_xcols))
    return Z

In [22]:
%timeit matrix_product2(X, Y)

31.3 ms ± 26 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
from faster_python_faster import matrix_product3

In [24]:
%timeit matrix_product3(X, Y)

21.2 ms ± 23.4 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
%lprun -f matrix_product3 bench(matrix_product3)

Timer unit: 1e-06 s

Total time: 1.43191 s
File: /home/leo/ME/computer-science/CSC Python/faster_python_faster.py
Function: matrix_product3 at line 66

Line #      Hits         Time  Per Hit   % Time  Line Contents
    66                                           def matrix_product3(X, Y):
    67        16         35.0      2.2      0.0      n_xrows, n_xcols = X.shape
    68        16         18.0      1.1      0.0      n_yrows, n_ycols = Y.shape
    69        16        217.0     13.6      0.0      Z = Matrix.zeros((n_xrows, n_ycols))
    70        16      21840.0   1365.0      1.5      Yt = Y.transpose()
    71      1040        553.0      0.5      0.0      for i, (Xi, Zi) in enumerate(zip(X, Z)):
    72     66560      28992.0      0.4      2.0          for k, Ytk in enumerate(Yt):
    73     65536      30892.0      0.5      2.2              Zi[k] = sum(Xi[j] * Ytk[j]
    74     65536    1349361.0     20.6     94.2                         for j in range(n_xcols))

In [26]:
import numpy as np

In [27]:
%timeit -n 100 matrix_product3(X, Y)

21.6 ms ± 307 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
X = np.random.randint(-255, 255, shape)
Y = np.random.randint(-255, 255, shape)

In [35]:
%timeit -n 100 X.dot(Y)

159 µs ± 37.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Numba

In [44]:
import numba

In [45]:
@numba.jit
def jit_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [47]:
%timeit -n 100 jit_matrix_product(X, Y)

162 µs ± 32.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [48]:
jit_matrix_product.inspect_types()

jit_matrix_product (array(int64, 2d, C), array(int64, 2d, C))
--------------------------------------------------------------------------------
# File: <ipython-input-45-d46f11eb1b2f>
# --- LINE 1 --- 

@numba.jit

# --- LINE 2 --- 

def jit_matrix_product(X, Y):

    # --- LINE 3 --- 
    # label 0
    #   X = arg(0, name=X)  :: array(int64, 2d, C)
    #   Y = arg(1, name=Y)  :: array(int64, 2d, C)
    #   $4load_attr.1 = getattr(value=X, attr=shape)  :: UniTuple(int64 x 2)
    #   $6unpack_sequence.4 = exhaust_iter(value=$4load_attr.1, count=2)  :: UniTuple(int64 x 2)
    #   del $4load_attr.1
    #   $6unpack_sequence.2 = static_getitem(value=$6unpack_sequence.4, index=0, index_var=None)  :: int64
    #   $6unpack_sequence.3 = static_getitem(value=$6unpack_sequence.4, index=1, index_var=None)  :: int64
    #   del $6unpack_sequence.4
    #   n_xrows = $6unpack_sequence.2  :: int64
    #   del $6unpack_sequence.2
    #   n_xcols = $6unpack_sequence.3  :: int64
    #   del $6unpack_seq

In [49]:
@numba.jit
def jit_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            Z[i, k] = (X[i, :] * Y[:, k]).sum()
    return Z

In [51]:
%timeit -n 100 jit_matrix_product(X, Y)

451 µs ± 74 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Cython

In [52]:
%load_ext cython

In [53]:
%%cython
print("Hello, world!")

Hello, world!


In [54]:
%%cython
def f():
    return 42
def g():
    return []

In [55]:
f

<function _cython_magic_5a901ab708a7ab0cc81eaa73f136a442.f>

In [56]:
g

<function _cython_magic_5a901ab708a7ab0cc81eaa73f136a442.g>

In [57]:
f(), g()

(42, [])

In [63]:
%%cython -a
from faster_python_faster import Matrix

def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()
    for i, (Xi, Zi) in enumerate(zip(X, Z)):
        for k, Ytk in enumerate(Yt):
            Zi[k] = sum(Xi[j] * Ytk[j] for j in range(n_xcols))
    return Z

In [62]:
%timeit cy_matrix_product(X, Y)

92.9 ms ± 442 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [65]:
%%cython -a
import numpy as np

def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    Yt = Y.transpose()
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [66]:
%timeit cy_matrix_product(X, Y)

136 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [67]:
%%cython -a
import numpy as np
cimport numpy as np

def cy_matrix_product(np.ndarray X, np.ndarray Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray Z
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [68]:
%timeit cy_matrix_product(X, Y)

133 ms ± 913 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [71]:
%%cython -a
import numpy as np
cimport numpy as np

def cy_matrix_product(
    np.ndarray[np.int64_t, ndim=2] X,
    np.ndarray[np.int64_t, ndim=2] Y
):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [72]:
%timeit cy_matrix_product(X, Y)

184 µs ± 692 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [73]:
%%cython -a
import numpy as np
cimport numpy as np
cimport cython


@cython.boundscheck(False)
@cython.overflowcheck(False)
def cy_matrix_product(
    np.ndarray[np.int64_t, ndim=2] X,
    np.ndarray[np.int64_t, ndim=2] Y
):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [74]:
%timeit cy_matrix_product(X, Y)

165 µs ± 2.52 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [75]:
%timeit -n 100 X.dot(Y)

155 µs ± 14.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
